In [2]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import math
import os
print(os.getcwd())


c:\Users\Joshualevy\Documents\capitalisntDashboard


In [3]:
#### 
def most_recent_collection(year: str, month: str, date: str) -> pd.DataFrame:
    """A function for reading in a .csv which regularly changes name.

    :param year: A string. Should be in %Y format (four-character year, eg. '2022')
    :param month: A string. Should be in %m format (two-character month, eg. '02')
    :param date: A string. Should be in %d format (two character date, eg. '09')
    
    :return df: A Pandas DataFrame containing episode-location-date level observations.
    """
    cities_path_base = 'us_cities_episode_locations-{yyyy}-{mm}-{dd}.csv'.format(
        yyyy = year,
        mm = month,
        dd = date
    )

    df = pd.read_csv(cities_path_base)
    return df




In [4]:
def generate_interp_dates(start_date: str, end_date: str) -> list:
    """A function for generating dates that were not collected by `apiCollect.py`

    Given the last date of legitimate collection prior to a period of absence and 
    the next date of legitimate collection following that absence, this function 
    returns a list of the dates in between.

    :param start_date: A string. Should be in '%Y-%m-%d' format. This should be 
    the last date of legitimate collection before there was a failed collection(s)
    (for whatever reason).
    :param end_date: A string. Should be in '%Y-%m-%d' format. This should be the
    first date of legitimate collection after there was a failed collection(s)
    (for whatever reason).

    :return interp_dates_list: A list. This contains a list of '%Y-%m-%d'-formatted
    dates. It does NOT include `start_date` or `end_date`.
    """

    start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    delta = end_date_dt - start_date_dt

    if delta.days <= 1:
        raise Exception("Hey, I think your dates don't require interpolation!")

    interp_dates_list = []
    for i in range(1, delta.days):
        new_day = start_date_dt + timedelta(days=i)
        new_day_str = datetime.strftime(new_day, '%Y-%m-%d')
        interp_dates_list.append(new_day_str)
        


    return interp_dates_list


In [5]:
def construct_interpolate_df(df: pd.DataFrame, start_interp_date: str, stop_interp_date) -> pd.DataFrame:
    """Given a DataFrame with missing dates, this function identifies the missing dates and re-sorts the columns to adhere to chronological order (important for interpolation)

    :param df: A DataFrame. This should be a DataFrame that is missing some date-columns. Those missing date columns are eventually interpolated.
    :param start_interp_date: A string. Should be in '%Y-%m-%d' format. This should be 
    the last date of legitimate collection before there was a failed collection(s)
    (for whatever reason).
    :param stop_interp_date: A string. Should be in '%Y-%m-%d' format. This should be the
    first date of legitimate collection after there was a failed collection(s)
    (for whatever reason).

    :return out_df: A DataFrame. This contains all date-columns (including the dates for which collection did not occur), properly sorted for interpolation.
    """
    
    dates_to_interp_list = generate_interp_dates(start_interp_date, stop_interp_date)


    out_df = df.copy()
    out_df = out_df.set_index(['city_name', 'city_id', 'state_id', 'episode_id'])
    out_df[dates_to_interp_list] = np.nan

    date_cols = list(out_df.columns)

    for index, date_str in enumerate(date_cols):
        if date_str == start_interp_date:
            split_open_index = index + 1
            continue
        elif date_str == stop_interp_date:
            split_close_index = index
            break

    pre_interp_dates_list = date_cols[0 : split_open_index]
    post_interp_dates_list = date_cols[split_close_index : -len(dates_to_interp_list)]

    ordered_interp_date_cols = pre_interp_dates_list + dates_to_interp_list + post_interp_dates_list
    out_df = out_df[ordered_interp_date_cols]

    out_df = out_df.interpolate(
        method = 'linear',
        axis = 'columns',
        limit_direction = 'forward',
        limit_area = 'inside'
    )

    return out_df



In [6]:
def interp_df_rebiasing(df: pd.DataFrame, start_interp_date: str, stop_interp_date: str) -> pd.DataFrame:
    """Uses linear interpolation to fill in missing daily downloads data for missing episode-location observations.

    This function actually fills in the data and then re-biases it to account for the logarithmic shape of downloads (generally). The first stage is to use a naive linear interpolation between the two dates that are observed. The second stage is to re-bias in favor of downloads closer to release date (see `linear_interp_biasing` for more.)

    :param df: A DataFrame. This should be a DataFrame that is missing some date-columns. Those missing date columns are eventually interpolated.
    :param start_interp_date: A string. Should be in '%Y-%m-%d' format. This should be 
    the last date of legitimate collection before there was a failed collection(s)
    (for whatever reason).
    :param stop_interp_date: A string. Should be in '%Y-%m-%d' format. This should be the
    first date of legitimate collection after there was a failed collection(s)
    (for whatever reason).

    :return df: A DataFrame that has new data filled in. This should have its index reset and then saved.
    """

    dates_to_rebias_list = generate_interp_dates(start_interp_date, stop_interp_date)

    if len(dates_to_rebias_list) == 1:
        interest_date = dates_to_rebias_list[0]
        df[interest_date] = df[[start_interp_date, interest_date, stop_interp_date]].apply(
            lambda obs: linear_interp_biasing(obs)
        )
    else:
        for index, interest_date in enumerate(dates_to_rebias_list):
            if index == 0:
                df[interest_date] = df[[start_interp_date, interest_date, dates_to_rebias_list[index + 1]]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )
            elif index == len(dates_to_rebias_list) - 1:
                df[interest_date] = df[[dates_to_rebias_list[index - 1], interest_date, stop_interp_date]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )
            else:
                df[interest_date] = df[[dates_to_rebias_list[index - 1], interest_date, dates_to_rebias_list[index + 1]]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )

    return df


    

In [7]:
def linear_interp_biasing(observation : pd.Series) -> float:
    """Re-biases download-data to biasing downloads toward release date (to mirror the generally logarithmic shape of daily downloads).

    :param observation: A Pandas series object. This Series object should have three elements indexed:
    0: the downloads on the day prior to the date of interest (as interpolated and potentially re-biased);
    1: the downloads of the date of interest (as interpolated and potentially re-biased);
    2: the downloads on the day after the date of (as interpolated and potentially re-biased)

    We re-bias because we do not want decimal daily downloads, an artefact that may be generated by the linear interpolation process. In the case that there are decimal downloads observed, we round up to the closest integer of downloads in the *next* day, and then round the next day's downloads down (or kept the same). This has the effect of moving downloads earlier in time. We do this to mimic the generally logarithmic shape of downloads over time and resolve the decimal problem.

    Because we process the columns of interest from left to right (from earliest to latest), this rounding/biasing montonically shifts biases downloads earlier in time.

    (pre, interest, post)
    Eg: (34, 34.5, 35) --> (34, 35, 35)
    Eg: (867, 868.5, 870) --> (867, 870, 870)

    :return interest: the re-biased downloads for the date of interest.
    """

    pre = observation.iloc[0]
    interest = observation.iloc[1]
    post = observation.iloc[2]

    if pre == interest:
        return interest
    
    if pre > interest:
        return pre
    elif interest.is_integer():
        return interest


    if pre < interest:
        if not interest.is_integer():
            # print((pre, interest, post))
            # print('NEW VALUE SHOULD BE: {}'.format(math.ceil(post)))
            return math.ceil(post)
            
    return interest



In [8]:
filename_year = '2022'
filename_month = '09'
filename_date = '20'

missing_df = most_recent_collection(filename_year, filename_month, filename_date)


new_df = missing_df.copy()
new_df = construct_interpolate_df(new_df, '2022-09-17', '2022-09-20')
new_df = interp_df_rebiasing(new_df, '2022-09-17', '2022-09-20')
new_df = new_df.reset_index(drop=False)
new_df.to_csv('us_cities_episode_locations-{}-{}-{}-INTERPOLATED.csv'.format(
    filename_year, filename_month, filename_date),
    index=False,
    encoding='utf-8'
)
new_df

NameError: name 'year' is not defined